# Quality Control for the Caudate Nucleus

In [ ]:
suppressMessages({library(repr)
                  library(ggpubr)
                  library(tidyverse)
                  library(moderndive)})

## Functions

In [ ]:
save_img <- function(image, fn, w, h){
    for(ext in c(".svg", ".pdf", ".png")){
        ggsave(file=paste0(fn, ext), plot=image, width=w, height=h)
    }
}

get_pheno <- function(){
    phenotypes <- data.table::fread(paste0("/ceph/projects/v4_phase3_paper/inputs/",
                                           "phenotypes/_m/merged_phenotypes.csv")) %>% 
        mutate_if(is.character, as.factor)
    return(phenotypes)
}

memPHENO <- memoise::memoise(get_pheno)

## Covariates exmination

### Examine if there are significant differences between diagnosis in continous covariates

In [ ]:
covarsCont = memPHENO() %>%
    select(-c('RNum', "Race", 'Sex', "Dx", 'Region', 'BrNum', 
              "antipsychotics", "lifetime_antipsych", "Protocol")) %>% colnames
options(repr.plot.width=12, repr.plot.height=6)
dir.create("covariate_plots")
for(covar in covarsCont){
    flush.console()
    bxp = memPHENO() %>% 
        filter(Age > 17, Race %in% c("AA", "EA"), Dx %in% c("CTL", "SZ")) %>%
        ggboxplot(x='Dx', y=covar, fill='Dx', add='jitter', palette="npg", 
                  facet.by="Region", legend='bottom', 
                  panel.labs.font=list(face='bold', size=14),
                  add.params=list(alpha=0.75)) + 
        stat_compare_means(comparisons=list(c("CTL", "SZ")), 
                           aes(label=..p.signif..), method="wilcox.test") +
        font("title", size=18, color="black", face="bold") + 
        font("xy.title", size=16, face="bold") + font("xy.text", size=14) + 
        font("legend.text", size=14) + font("legend.title", size=16, face="bold")
    save_img(bxp, paste0("covariate_plots/boxplot_dx_filtered_",covar), w=12, h=4)
    print(bxp)
    bxp = memPHENO() %>% 
        filter(Age > 17, Race %in% c("AA", "EA"), Dx %in% c("CTL", "SZ")) %>%
        ggboxplot(x='antipsychotics', y=covar, color='Dx', add='jitter', 
                    palette="npg", facet.by="Region", legend='bottom') + 
        stat_compare_means(comparisons=list(c("TRUE", "FALSE")), 
                           aes(label=..p.signif..), method="wilcox.test") +
        font("title", size=18, color="black", face="bold") + 
        font("xy.title", size=16, face="bold") + font("xy.text", size=14) + 
        font("legend.text", size=14) + font("legend.title", size=16, face="bold")
    save_img(bxp, paste0("covariate_plots/boxplot_dxBYantipsychotics_filtered_",covar), 
             w=12, h=4)
    print(bxp)
    bxp = memPHENO() %>% 
        ggboxplot(x='Dx', y=covar, fill='Dx', add='jitter', palette="npg", 
                  facet.by="Region", legend='bottom', 
                  panel.labs.font=list(face='bold', size=14),
                  add.params=list(alpha=0.75)) + 
        stat_compare_means(comparisons=list(c("CTL", "SZ")), 
                           aes(label=..p.signif..), method="wilcox.test") +
        font("title", size=18, color="black", face="bold") + 
        font("xy.title", size=16, face="bold") + font("xy.text", size=14) + 
        font("legend.text", size=14) + font("legend.title", size=16, face="bold")
    print(bxp)
    save_img(bxp, paste0("covariate_plots/boxplot_dx_all_",covar), w=12, h=4)
}

## Examine gene expression relating to covariates

### Load TPM (log2)

In [ ]:
caudate = data.table::fread("/ceph/projects/v4_phase3_paper/inputs/counts/text_files_counts/tpm/_m/caudate/gene/log2tpm.csv") %>% 
    column_to_rownames(var="names") %>% t %>% as.data.frame 
dlpfc = data.table::fread("/ceph/projects/v4_phase3_paper/inputs/counts/text_files_counts/tpm/_m/dlpfc/gene/log2tpm.csv") %>% 
    column_to_rownames(var="names") %>% t %>% as.data.frame 
hippo = data.table::fread("/ceph/projects/v4_phase3_paper/inputs/counts/text_files_counts/tpm/_m/hippocampus/gene/log2tpm.csv") %>% 
    column_to_rownames(var="names") %>% t %>% as.data.frame 

### Dimensional reduction

In [ ]:
pca_cc = prcomp(caudate, center = TRUE)$x
pca_dd = prcomp(dlpfc, center = TRUE)$x
pca_hh = prcomp(hippo, center = TRUE)$x

### Merge tissues

In [ ]:
dtc = pca_cc %>% as.data.frame %>% rownames_to_column() %>% 
    select(c(rowname, PC1, PC2, PC3, PC4, PC5)) %>% 
    pivot_longer(-rowname, names_to="PC", values_to="PC_values") %>% 
    mutate(tissue="Caudate")
dtd = pca_dd %>% as.data.frame %>% rownames_to_column() %>% 
    select(c(rowname, PC1, PC2, PC3, PC4, PC5)) %>% 
    pivot_longer(-rowname, names_to="PC", values_to="PC_values") %>% 
    mutate(tissue="DLPFC")
dth = pca_hh %>% as.data.frame %>% rownames_to_column() %>% 
    select(c(rowname, PC1, PC2, PC3, PC4, PC5)) %>% 
    pivot_longer(-rowname, names_to="PC", values_to="PC_values") %>% 
    mutate(tissue="Hippocampus")
dt = bind_rows(dtc, dtd, dth)

### Plot scatters and correlations with covariates

In [ ]:
colors = get_palette(palette = "npg", 3)
colors

In [ ]:
options(repr.plot.width=18, repr.plot.height=9)
dir.create("pca_scatter")
for(covar in covarsCont){
    flush.console()
    sca1 = dt %>% inner_join(memPHENO(), by=c("rowname"="RNum")) %>% 
        ggscatter(y="PC_values", x=covar, color="Dx", palette="npg",
                  facet.by=c('tissue', 'PC'), ncol=5,
                  add='reg.line', conf.int=TRUE, cor.coef=TRUE,
                  add.params=list(color="blue", fill="lightgray"))
    save_img(sca1, paste0("pca_scatter/scatter_log2tpm_dx_5pcs_",covar), w=18, h=9)
    print(sca1)
    sca2 = dt %>% inner_join(memPHENO(), by=c("rowname"="RNum")) %>% 
        filter(antipsychotics == "TRUE", Dx == "SZ") %>%
        ggscatter(y="PC_values", x=covar, color=colors[2], facet.by=c('tissue', 'PC'), 
                  ncol=5, add='reg.line', conf.int=TRUE, cor.coef=TRUE,
                  palette="npg", add.params=list(color="blue", fill="lightgray"))
    save_img(sca2, paste0("pca_scatter/scatter_log2tpm_AP_5pcs_",covar), w=18, h=9)
    print(sca2)
    sca3 = dt %>% inner_join(memPHENO(), by=c("rowname"="RNum")) %>% 
        filter(antipsychotics == "FALSE", Dx == "SZ") %>%
        ggscatter(y="PC_values", x=covar, color=colors[1], facet.by=c('tissue', 'PC'), 
                  ncol=5, add='reg.line', conf.int=TRUE, cor.coef=TRUE,
                  palette="npg", add.params=list(color="blue", fill="lightgray"))
    save_img(sca3, paste0("pca_scatter/scatter_log2tpm_noAP_5pcs_",covar), w=18, h=9)
    print(sca3)
}

In [ ]:
rm(caudate, dlpfc, dtc, dtd, dth, hippo)

## Test covariates with Caudate

In [ ]:
suppressMessages({library(SummarizedExperiment)
                  library(limma)
                  library(edgeR)
                  library(sva)})

# Function from jaffelab github
merge_rse_metrics <- function(rse) {
    stopifnot(is(rse, 'RangedSummarizedExperiment'))
    stopifnot(
        c('concordMapRate', 'overallMapRate', 'mitoRate', 'rRNA_rate',
          'totalAssignedGene', 'numMapped', 'numReads', 'numUnmapped',
          'mitoMapped', 'totalMapped') %in%
            colnames(SummarizedExperiment::colData(rse))
    )

    stopifnot(all(sapply(c(
        'concordMapRate', 'overallMapRate', 'mitoRate', 'rRNA_rate',
        'totalAssignedGene', 'numMapped', 'numReads', 'numUnmapped',
        'mitoMapped', 'totalMapped'), function(var) {
            is(colData(rse)[, var], 'List')
        })
    ))

    rse$concordMapRate = mapply(function(r, n) {
        sum(r*n)/sum(n)
    }, rse$concordMapRate, rse$numReads)
    rse$overallMapRate = mapply(function(r, n) {
        sum(r*n)/sum(n)
    }, rse$overallMapRate, rse$numReads)
    rse$mitoRate = mapply(function(r, n) {
        sum(r*n)/sum(n)
    }, rse$mitoRate, rse$numMapped)
    rse$rRNA_rate = mapply(function(r, n) {
        sum(r*n)/sum(n)
    }, rse$rRNA_rate, rse$numMapped)
    rse$totalAssignedGene = mapply(function(r, n) {
        sum(r*n)/sum(n)
    }, rse$totalAssignedGene, rse$numMapped)

    rse$numMapped = sapply(rse$numMapped, sum)
    rse$numReads = sapply(rse$numReads, sum)
    rse$numUnmapped = sapply(rse$numUnmapped, sum)
    rse$mitoMapped = sapply(rse$mitoMapped, sum)
    rse$totalMapped = sapply(rse$totalMapped, sum)
    return(rse)
}

In [ ]:
library(memoise)
prep_data <- function(count_fn){
    mds_file = "/ceph/projects/v4_phase3_paper/inputs/genotypes/mds/_m/LIBD_Brain_TopMed.mds"
    mds = data.table::fread(mds_file) %>% 
        rename("C1"="snpPC1", "C2"="snpPC2", "C3"="snpPC3", 
               "C4"="snpPC4", "C5"="snpPC5") %>%
        mutate_if(is.character, as.factor)
    load(count_fn)
    keepIndex = which((rse_gene$Dx %in% c("Control", "Schizo")) & 
                      rse_gene$Age > 17 & rse_gene$Race %in% c("AA", "CAUC"))
    rse_gene = rse_gene[, keepIndex]
    rse_gene$Dx = factor(rse_gene$Dx, levels = c("Control", "Schizo"))
    rse_gene$Sex <- factor(rse_gene$Sex)
    rse_gene <- merge_rse_metrics(rse_gene)
    rse_gene$ERCCsumLogErr <- mapply(function(r, n) {
        sum(r * n)/sum(n)
    }, rse_gene$ERCCsumLogErr, rse_gene$numReads)
    colData(rse_gene)$RIN = sapply(colData(rse_gene)$RIN,"[",1)
    pheno = colData(rse_gene) %>% as.data.frame %>% 
        inner_join(mds, by=c("BrNum"="FID")) %>% 
        distinct(RNum, .keep_all = TRUE) 
    # Generate DGE list
    x <- DGEList(counts=assays(rse_gene)$counts[,pheno$RNum], 
                 genes=rowData(rse_gene), samples=pheno)
    # Filter by expression
    design0 <- model.matrix(~Dx, data=x$samples)
    keep.x <- filterByExpr(x, design=design0)
    x <- x[keep.x, , keep.lib.sizes=FALSE]
    print(paste('There are:', sum(keep.x), 'features left!', sep=' '))
    # Normalize library size
    x <- calcNormFactors(x, method="TMM")
    return(x)
}

memo_prepData <- memoise(prep_data)

qsv_model <- function(count_fn, qsv_file){
    x <- memo_prepData(count_fn)
    # Design matrix
    mod = model.matrix(~Dx + Age + Sex + mitoRate + rRNA_rate + 
                       totalAssignedGene + RIN + overallMapRate + 
                       snpPC1 + snpPC2 + snpPC3, data = x$samples)
    colnames(mod) <- gsub("Dx", "", colnames(mod))
    colnames(mod) <- gsub("SexM", "Male", colnames(mod))
    colnames(mod) <- gsub("\\(Intercept\\)", "Intercept", colnames(mod))
    # qSV
    modQsva <- mod %>% as.data.frame %>% rownames_to_column() %>%
        inner_join(data.table::fread(qsv_file), by=c("rowname"="V1")) %>% 
        rename_all(list(~str_replace_all(., 'PC', 'qPC'))) %>% 
        column_to_rownames("rowname") %>% as.matrix
    return(modQsva)
}

memo_qsvModel <- memoise(qsv_model)

pca_select <- function(count_fn){
    ### Dimensional reduction (PCA)
    x <- memo_prepData(count_fn)
    log2cpm = cpm(x, log=TRUE) %>% t
    pca_df = prcomp(log2cpm, center=TRUE)$x
    dt = pca_df %>% as.data.frame %>% rownames_to_column() %>% 
        select(c(rowname, PC1, PC2, PC3, PC4, PC5)) %>% 
        pivot_longer(-rowname, names_to="PC", values_to="PC_values")
    return(dt)
}

memo_pcaSelect <- memoise(pca_select)

save_norm <- function(count_fn){
    x <- memo_prepData(count_fn)
    return(cpm(x, log=TRUE))
}

memNORM <- memoise(save_norm)

get_voom <- function(count_fn, qsv_file){
    ### Preform voom
    x <- memo_prepData(count_fn)
    modQsva <- memo_qsvModel(count_fn, qsv_file)
    v <- voom(x, modQsva, plot=FALSE)
    return(v)
}

memo_voom <- memoise(get_voom)

cal_res <- function(count_fn, qsv_file){
    ### Calculate residuals
    v <- memo_voom(count_fn, qsv_file)
    null_model <- v$design %>% as.data.frame %>% select(-c("Schizo")) %>% as.matrix
    fit_res <- lmFit(v, design=null_model)
    res = v$E - ( fit_res$coefficients %*% t(null_model) )
    res_sd = apply(res, 1, sd)
    res_mean = apply(res, 1, mean)
    res_norm = (res - res_mean) / res_sd
    return(res_norm)
}

memo_res <- memoise(cal_res)

## Normalize residuals
pca_res <- function(count_fn, qsv_file){
    res_df = memo_res(count_fn, qsv_file) %>% t
    pca_df = prcomp(res_df, center=TRUE)$x
    dt = pca_df %>% as.data.frame %>% rownames_to_column %>%
        select(c(rowname, PC1, PC2, PC3, PC4, PC5)) %>% 
        pivot_longer(-rowname, names_to="PC", values_to="PC_values")
    return(dt)
}

memo_pcaRES <- memoise(pca_res)

## Differential Expression Analysis

In [ ]:
counts_lt = list("Caudate"=paste0("/ceph/projects/v3_phase3_paper/inputs/phase3/_m/count_data/", 
                                  "caudate_brainseq_phase3_hg38_rseGene_merged_n464.rda"), 
                 "DLPFC"=paste0("/ceph/projects/v3_phase3_paper/inputs/phase2/_m/count_data/", 
                                "dlpfc_ribozero_brainseq_phase2_hg38_rseGene_merged_n453.rda"), 
                 "HIPPO"=paste0("/ceph/projects/v3_phase3_paper/inputs/phase2/_m/count_data/", 
                                "hippo_brainseq_phase2_hg38_rseGene_merged_n447.rda"))
qsv_lt = list("Caudate"="/ceph/projects/v4_phase3_paper/inputs/counts/text_files_counts/_m/caudate/qSV_caudate.csv", 
              "DLPFC"="/ceph/projects/v4_phase3_paper/inputs/counts/text_files_counts/_m/dlpfc/qSV_dlpfc.csv", 
              "HIPPO"="/ceph/projects/v4_phase3_paper/inputs/counts/text_files_counts/_m/hippocampus/qSV_hippo.csv")

### Plot scatters and correlations with covariates

In [ ]:
for(tissue in c('Caudate', 'DLPFC', 'HIPPO')){
    flush.console()
    lotissue = paste0(tolower(tissue), "/")
    dir.create(lotissue)
    options(repr.plot.width=18, repr.plot.height=6)
    modQsva <- memo_qsvModel(counts_lt[[tissue]], qsv_lt[[tissue]])
    dt <- memo_pcaSelect(counts_lt[[tissue]])
    covar_model = modQsva %>% as.data.frame %>% rownames_to_column 
    cols = modQsva %>% as.data.frame %>% 
        select(-c(Intercept, Male, Schizo)) %>% colnames
    for(covar in cols){
        flush.console()
        sca = dt %>% inner_join(covar_model, by="rowname") %>% 
            inner_join(memPHENO()[, c("RNum", "Sex", "Dx")], by=c("rowname"="RNum")) %>%
            ggscatter(y="PC_values", x=covar, color="Dx", facet.by=c('PC'), ncol=5, 
                      add='reg.line', conf.int=TRUE, cor.coef=TRUE, palette="npg",
                      add.params=list(color="blue", fill="lightgray"))
        save_img(sca, paste0(lotissue,"scatter_log2cpm_dx_5pcs_",covar), w=18, h=6)
        print(sca)
    }
    cols = modQsva %>% as.data.frame %>% 
        select(-c(Intercept, Male, Schizo)) %>% colnames
    memNORM(counts_lt[[tissue]]) %>% as.data.frame %>%
        data.table::fwrite(paste0(lotissue, "normalized_expression.tsv"), 
                                  sep='\t', row.names=TRUE)
    memo_res(counts_lt[[tissue]], qsv_lt[[tissue]]) %>% as.data.frame %>% 
        data.table::fwrite(paste0(lotissue, "residualized_expression.tsv"), 
                           sep='\t', row.names=TRUE)
    dt <- memo_pcaRES(counts_lt[[tissue]], qsv_lt[[tissue]])
    for(covar in cols){
        flush.console()
        sca = dt %>% inner_join(covar_model, by="rowname") %>% 
            inner_join(memPHENO()[, c("RNum", "Sex", "Dx")], by=c("rowname"="RNum")) %>%
            ggscatter(y="PC_values", x=covar, color="Dx", facet.by=c('PC'), ncol=5, 
                      add='reg.line', conf.int=TRUE, cor.coef=TRUE, palette="npg",
                      add.params=list(color="blue", fill="lightgray"))
        save_img(sca, paste0(lotissue, "scatter_normres_dx_5pcs_",covar), w=18, h=6)
        print(sca)
    }
}

## Reproducibility Information

In [ ]:
Sys.time()
proc.time()
options(width = 120)
sessioninfo::session_info()